In [1]:
from phisher.xai import GradCAM
from phisher.module import PhisherhModule
from phisher.model import PhisherEmbeddingModel
from phisher.dataset import PhishingEmbeddingDataset

/Users/bkosinski/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch

#### Load model

In [3]:
model_ckpt_path: str = "/Users/bkosinski/Desktop/STUDIA/SEMI/GSN/projekt/checkpoints/embeddings_model.ckpt"

In [4]:
model: PhisherEmbeddingModel = PhisherEmbeddingModel(vocab_size=84, embedding_dim=100, out_features=1)

In [5]:
PhisherhModule.load_from_checkpoint(model_ckpt_path, model=model)

/Users/bkosinski/Library/Python/3.9/lib/python/site-packages/lightning_fabric/utilities/cloud_io.py:55: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
/Users/bkosinski/Library/Python/3.9/lib/

PhisherhModule(
  (model): PhisherEmbeddingModel(
    (embedding): Embedding(84, 100, padding_idx=0)
    (conv1): Conv2d(1, 6, kernel_size=(5, 1), stride=(1, 1))
    (conv2): Conv2d(6, 12, kernel_size=(5, 1), stride=(1, 1))
    (fc1): Linear(in_features=56400, out_features=120, bias=True)
    (fc2): Linear(in_features=120, out_features=60, bias=True)
    (out): Linear(in_features=60, out_features=1, bias=True)
  )
  (accuracy): BinaryAccuracy()
  (f1): BinaryF1Score()
  (precision): BinaryPrecision()
  (recall): BinaryRecall()
  (auroc): BinaryAUROC()
)

#### Visualize Grad-CAM

In [6]:
def visualize_url_with_cam(url: str, cam: torch.Tensor) -> str:
    cam = cam.squeeze()
    if len(cam.shape) != 1:
        cam = cam.mean(dim=1)
        cam = (cam - cam.min()) / (cam.max() - cam.min())
    colored_url = ""
    for char, intensity in zip(url, cam):
        color_intensity = int(
            255 * (1 - intensity)
        )
        hex_color = f"{color_intensity:02x}{color_intensity:02x}ff"
        colored_url += f'<span style="background-color: #{hex_color};">{char}</span>'

    return colored_url

In [7]:
def parse_url_to_input_tensor(url: str, placeholder_dataset: PhishingEmbeddingDataset, device: str) -> torch.Tensor:
    input_tensor = placeholder_dataset.parse_url(url)
    input_tensor = placeholder_dataset.pad_or_trim(input_tensor)
    input_tensor = torch.tensor(input_tensor, dtype=torch.long)

    input_tensor = input_tensor.to(device)
    input_tensor = input_tensor.reshape(1, -1)
    return input_tensor

In [8]:
device = torch.device("mps")
model = model.to(device)

In [9]:
grad_cam = GradCAM(model, target_layer=model.conv1)

In [10]:
placeholder_csv_file_path: str = "/Users/bkosinski/Desktop/STUDIA/SEMI/GSN/projekt/phish_dataset.csv"
placeholder_dataset = PhishingEmbeddingDataset(csv_file_path=placeholder_csv_file_path)

In [47]:
target_url: str = "apple.tracker-liveview.support" # this is a phishing URL, see -> https://hole.cert.pl/domains/v2/domains.txt

In [48]:
input_tensor = parse_url_to_input_tensor(target_url, placeholder_dataset, device)
cam = grad_cam.generate_cam(input_tensor, target_class=0)

In [49]:
output = model(input_tensor)
probs = torch.sigmoid(output.squeeze(1))
preds = (probs >= 0.5).int()

In [50]:
class_mapping: dict = {
    0: "Phishing",
    1: "Safe"
}

In [51]:
visualized_url: str = visualize_url_with_cam(url=target_url, cam=cam)

In [52]:
from IPython.core.display import display, HTML
display(HTML(f"<div style='color: black'>{visualized_url}</div>"))

/var/folders/l3/hg12zsjx6rsbsylxwh747cv00000gn/T/ipykernel_52826/1759908886.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [160]:
print(f"Predicted class is: {class_mapping[preds.item()]}")

Predicted class is: Phishing
